# smooth data and fit first level GLM

K. Garner, 2022  
This code will smooth the data and fit a first level GLM to each participant's fmri data.  
Assumptions:  
Data is BIDS enough.  


## set up environment

In [1]:
from bids.layout import BIDSLayout
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms import modelgen as mgen
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

220428-17:12:45,284 nipype.utils WARNING:
	 A newer version (1.7.1) of nipy/nipype is available. You are using 1.5.0


## define source data location and establish workflow

In [2]:
basedir = '/scratch/qbi/uqkgarn1/data/'
layout = BIDSLayout(basedir)
subs = layout.get_subjects()

glm = pe.Workflow(name='glms') # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


## data grabbing

In [3]:
dg = pe.Node(DataGrabber(infields= ['sub'], 
                         outfields=['func', 'motion', 'onsets', 'bjson', 'mask']), 
                         name='data-grabber')
dg.inputs.base_dir = '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/'


dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'func': [['sub', 'sub']],
                           'motion':[['sub', 'sub']],
                           'onsets':[['sub', 'sub']],
                           'bjson':[['sub', 'sub']],
                           'mask':[['sub', 'sub']]}
dg.inputs.field_template = {'func':   '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/func/sub-%s_*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
                            'motion': '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/func/sub-%s_ses-02_task-attlearn_run-*_desc-motion_timeseries.txt',
                            'onsets': '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/beh/sub-%s_ses-02_task-attlearn_run-*_desc-glm-onsets.json',
                            'bjson':  '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/func/sub-%s_ses-02_task-attlearn_run-*_space-MNI152NLin2009cAsym_desc-preproc_bold.json',
                            'mask':   '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-%s/ses-02/anat/sub-%s_ses-02_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz'}

In [4]:
dg.inputs.sub = subs[0]
res = dg.run()
res.outputs

220428-17:12:47,689 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpqgz9o_ly/data-grabber".
220428-17:12:47,695 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
220428-17:12:47,702 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



bjson = ['/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.json', '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.json']
func = ['/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']
mask = /scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/anat/sub-01_ses-02_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz
motion = ['/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-attlearn_run-2_desc-motion_timeseries.txt', '/scratch/qbi/uqkgarn1/data/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-

In [5]:
info = pe.Node(ni.IdentityInterface(fields=['sub', 'sub']), name = 'info')
info.iterables = [('sub', ['01'])]

In [6]:
def printSubPath(full_file_path):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the 
    # container string
    # given the full filepath, this extracts the subj/ses folder names for input
    # into DataSink
    # Args
    # -- full_file_path: cell of file names, any one of the data grabber outputs should work
    # Returns
    # -- string of the form 'sub'0x/ses-0x'
    import os
    import re
    fname = os.path.normpath(full_file_path[0])
    l = fname.split(os.sep)
    sub = [s for s in l if re.search('sub', s)][0]
    ses = [s for s in l if re.search('ses', s)][0]
    name = [sub, ses]
    name = '/'.join(name)
    return name



## datasink node for data saves

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/data/derivatives/spm/"
substitutions = [('_sub_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

## get design info

In [8]:
def getOnsetsJson(input_files):
    # reads the onsets for GLM from each run, and puts together as a bunch, ready for use in spm/model def
    # Args:
    # -- input_files [cell list of filenames] output as 'onsets' from datagrabber
    # Returns
    # a Bunch of dim n runs, containing the fields 'conditions', 'onsets', 'durations'
    from nipype.interfaces.base import Bunch
    import json
    prt_output = [] #prt=protocol
    count = 0
    for f in input_files: 
        count = count + 1
        with open(f, "r+") as file:
            data = json.load(file)
            prt_output.insert(count, 
                              Bunch(conditions=data['names'],
                                    onsets=data['onsets'],
                                    durations=data['durations']))
    return prt_output

In [9]:
# now a node to use the getOnsetsJson function in the work flow
get_onsets = pe.Node(Function(input_names = ['input_files'],
                              output_names = ['prt_output'],
                              function = getOnsetsJson),
                     name = 'get_prt_onsets')

## some unzipping modules so that spm can handle the images

Note: I opted to create 2 to avoid output conflicts

In [10]:
gunzip_func = pe.MapNode(Gunzip(), name='gunzipfunc', iterfield=['in_file'])
gunzip_mask = pe.Node(Gunzip(), name='gunzipmask')

## smooth the functional data

In [11]:
smooth = pe.Node(spm.Smooth(), name='smooth')
smooth.inputs.fwhm = [2, 2, 2]  # play with this for sub 1 

## grab the TR for the model specification

In [12]:
def getTRJson(input_files):
    # extracts the TR in seconds from one of the json files accompanying the T2 images
    # Args
    # -- input_files: cell list of json files for bold images (bjson from data grabber)
    # Returns
    # -- TR [numeric]: TR is seconds
    import json
    with open(input_files[0], "r+") as file:
            data = json.load(file)
            TR = data['RepetitionTime'] 
    return TR

In [13]:
get_tr = pe.Node(Function(input_names=['input_files'],
                          output_names=['TR'],
                          function=getTRJson),
                 name='get_TR')

## specify glm, generate design matrix and estimate

In [14]:
model_spec = pe.Node(mgen.SpecifySPMModel(concatenate_runs=False,
                                         input_units='secs',
                                         output_units='secs',
                                         high_pass_filter_cutoff=128),
                     name="modelspec")

In [15]:
level_1_design = pe.Node(spm.Level1Design(bases={'hrf': {'derivs': [1, 1]}},
                                          timing_units='secs',
                                          model_serial_correlations='FAST',
                                          factor_info=[dict(name='tgt_side', levels=2),
                                                       dict(name='p', levels=2),
                                                       dict(name='val', levels=4)]),
                         name="level1design")

In [16]:
estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1},
                                     write_residuals=False),
                                     name="estimate")

## Connect workflow and run

In [ ]:
glm.connect([(info, dg, [('sub', 'sub')]), # get sub info to feed into data grabber
             (dg, ds, [(('motion', printSubPath), # print the path for saving data to (i.e. sub-0x/ses-0x)
                         'container')]),
             (dg, get_onsets, [('onsets', 'input_files')]), # get the onsets for each run into a single set of bunches
             (dg, gunzip_func, [('func', 'in_file')]), # unzip t2
             (dg, gunzip_mask, [('mask', 'in_file')]), # unzip masks
             (gunzip_func, smooth, [('out_file', 'in_files')]), # smooth data
             (smooth, ds, [('smoothed_files', 'data.@smooth')]), # save smoothed
             (dg, get_tr, [('bjson', 'input_files')]), # get TR for model definition
             (get_tr, model_spec, [('TR', 'time_repetition')]), # TR for model spec
             (dg, model_spec, [('motion', 'realignment_parameters')]),
             (get_onsets, model_spec, [('prt_output', 'subject_info')]),
             (smooth, model_spec, [('smoothed_files', 'functional_runs')]),
             (get_tr, level_1_design, [('TR', 'interscan_interval')]),
             (model_spec, level_1_design, [('session_info', 'session_info')]),
             (gunzip_mask, level_1_design, [('out_file', 'mask_image')]), 
             (level_1_design, estimate, [('spm_mat_file', 'spm_mat_file')]),
             (estimate, ds, [('beta_images', 'flglm.@beta')]),
             (estimate, ds, [('spm_mat_file', 'flglm.@des')]),
             (estimate, ds, [('residual_image', 'flglm.@res')]),
             (estimate, ds, [('mask_image', 'flglm.@mask')])
])
glm.run()          

220428-17:12:47,840 nipype.workflow INFO:
	 Workflow glms settings: ['check', 'execution', 'logging', 'monitoring']
220428-17:12:47,859 nipype.workflow INFO:
	 Running serially.
220428-17:12:47,860 nipype.workflow INFO:
	 [Node] Setting-up "glms.data-grabber" in "/tmp/tmpqgz9o_ly/data-grabber".
220428-17:12:47,861 nipype.workflow INFO:
	 [Node] Outdated cache found for "glms.data-grabber".
220428-17:12:47,866 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
220428-17:12:47,871 nipype.workflow INFO:
	 [Node] Finished "glms.data-grabber".
220428-17:12:47,872 nipype.workflow INFO:
	 [Node] Setting-up "glms.get_TR" in "/tmp/tmpsnxs_dw6/glms/_sub_01/get_TR".
220428-17:12:47,876 nipype.workflow INFO:
	 [Node] Running "get_TR" ("nipype.interfaces.utility.wrappers.Function")
220428-17:12:47,880 nipype.workflow INFO:
	 [Node] Finished "glms.get_TR".
220428-17:12:47,881 nipype.workflow INFO:
	 [Node] Setting-up "glms.gunzipmask" in "/tmp/tmpkcrqvwin/glms